# Fine-Tuning Open Source LLM for Customer Service Chatbot

In [1]:
# Imports
import os
import json
import torch
import evaluate
import torch.nn as nn
import transformers
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model
from datasets import Dataset, Features, ClassLabel, Value, Sequence
import warnings
warnings.filterwarnings('ignore')

c:\Users\krupc\Downloads\Projects\mlops\falcon-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Checking the GPU
if torch.cuda.is_available():
    print('Number of GPUs:', torch.cuda.device_count())
    print('GPU Model: ', torch.cuda.get_device_name(0))
    print('Total Memory [GB] of GPU: ', round(torch.cuda.get_device_properties(0).total_memory / 1e9, 1))

Number of GPUs: 1
GPU Model:  NVIDIA GeForce RTX 4060 Ti
Total Memory [GB] of GPU:  8.6
